# Question1

# Systematic Encoding Matrix Generator Function

This function is designed to receive a parity check matrix $H$ and build a systematic encoding matrix $G$ for it. The process involves potentially renaming the variables to ensure systematic encoding.

## Function Description

- **Input**: The function takes a parity check matrix $H$.
- **Processing**: It rearranges $H$ to $\hat{H}$, which is equal to $H$ up to a column permutation. Then, it constructs a systematic encoding matrix $G$.
- **Output**: The function returns two matrices:
  - $\hat{H}$: A rearranged version of $H$ after column permutation.
  - $G$: A systematic encoding matrix such that $\hat{H}Gt = 0$ in $F_{2}$ (the field of two elements).

## Requirements

- All operations must be performed in the finite field $F_{2}$ (binary field).
- The matrix $G$ should be constructed in a way that it satisfies the condition $\hat{H}Gt = 0$.

## Example

Consider testing the function with the following matrix $H$:

$H = \begin{bmatrix}
  1 & 1 & 1 & 1 & 0 & 0 \\
  0 & 0 & 1 & 1 & 0 & 1 \\
  1 & 0 & 0 & 1 & 1 & 0 
    \end{bmatrix}$

The function should return matrices $\hat{H}$ and $G$ for this input. Note that this function may also be tested with other input matrices.



In [1]:
import numpy as np
from scipy.linalg import lu

In [2]:
H = np.array([[1,1,1,1,0,0],
              [0,0,1,1,0,1],
              [1,0,0,1,1,0]],dtype=int)
print(H)

[[1 1 1 1 0 0]
 [0 0 1 1 0 1]
 [1 0 0 1 1 0]]


In [3]:
pl, u = lu(H, permute_l=True)
pl

array([[ 1.,  0.,  0.],
       [ 0., -0.,  1.],
       [ 1.,  1.,  0.]])

In [4]:
u

array([[ 1.,  1.,  1.,  1.,  0.,  0.],
       [ 0., -1., -1.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  1.]])

LU decomposition: https://www.youtube.com/watch?v=BFYFkn-eOQk

### Question 1

## Function Description

### Function Purpose
The function `generator_matrix` takes a parity check matrix $H$ and constructs a systematic encoding matrix $G$ for it.

### Parameters
- $H$: A parity check matrix, represented as a 2D array or matrix.

### Returns
- $\hat{H}$: A matrix equal to $H$ up to a column permutation.
- $G$: The systematic encoding matrix constructed from $H$.

### Constraints
- $H$ should be a valid non-empty matrix.

In [7]:
def generator_matrix(parity_check_matrix):
    
    def to_reduced_row_echelon_form(matrix):
        
        rows, cols = matrix.shape    
        pivot_row = pivot_col = row_to_swap = 0
        echelon_matrix = matrix.copy()

        while True:
            if pivot_row >= rows or pivot_col >= cols:
                break

            # If the current entry is zero, find a non-zero row to swap
            if echelon_matrix[pivot_row, pivot_col] == 0:
                row_to_swap = pivot_row
                while row_to_swap < rows and echelon_matrix[row_to_swap, pivot_col] == 0:
                    row_to_swap += 1

                # If no non-zero entry is found in the column, move to the next column
                if row_to_swap == rows:
                    pivot_col += 1
                    continue

                # Swap the rows
                echelon_matrix[pivot_row, :], echelon_matrix[row_to_swap, :] = echelon_matrix[row_to_swap, :].copy(), echelon_matrix[pivot_row, :].copy()

            # Row elimination to create zeros in all other rows of current column
            for current_row in range(rows):
                if current_row != pivot_row and echelon_matrix[current_row, pivot_col] != 0:
                    echelon_matrix[current_row, :] = (echelon_matrix[current_row, :] + echelon_matrix[pivot_row, :]) % 2

            # Move to the next element in the diagonal
            pivot_row += 1
            pivot_col += 1

        return echelon_matrix

    # Calculate the dimensions of the parity check matrix
    rows, cols = parity_check_matrix.shape

    # Calculate the dimension k of the code
    code_dimension = cols - rows

    # Convert the parity check matrix to its reduced row echelon form
    echelon_matrix = to_reduced_row_echelon_form(parity_check_matrix)

    # Directly construct the generator matrix
    generator_matrix = np.vstack((echelon_matrix[:, rows:cols], np.identity(code_dimension, dtype=int)))

    return echelon_matrix, generator_matrix


In [8]:
H_hat, G = generator_matrix(H)

print("H_hat:\n",H_hat, "\nG:\n", G)

H_hat:
 [[1 0 0 1 1 0]
 [0 1 0 1 1 1]
 [0 0 1 1 0 1]] 
G:
 [[1 1 0]
 [1 1 1]
 [1 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]


## LDPC Decoder Implementation Using Loopy Belief Propagation

### Task
Develop an LDPC decoder based on Loopy Belief Propagation suitable for a Binary Symmetric Channel.

### Function Specifications
- **Inputs**:
  - $\hat{H}$: (Parity Check Matrix)
  - $y$: (Received Word)
  - $p$: (Noise Ratio)
  - $max\_iterations$: (Optional, Maximum Number of Iterations; default value: 20)

- **Output**:
  - Decoded vector
  - Return code:
    - $0$: for success
    - $-1$: if the maximum number of iterations is reached without successful decoding


In [9]:
with open("H1.txt") as file_h:
    h1 = np.array([list(map(int, line.split())) for line in file_h if line.strip()], dtype=int)

print("h1 shape: " + str(h1.shape))

with open("y1.txt") as file_y:
    y1 = np.array([int(line.strip()) for line in file_y if line.strip()], dtype=int)

print("y1 shape: " + str(y1.shape))


h1 shape: (750, 1000)
y1 shape: (1000,)


In [10]:
def decode(parity_check_matrix, received_signal, noise_ratio=.1, maximum_iterations=20):

    decoding_success = -1
    num_rows, num_cols = parity_check_matrix.shape

    # List of factor checks
    bit_check_indices = []
    for row in range(num_rows):
        bit_check_indices.append(np.where(parity_check_matrix[row] == 1)[0])

    # Probability calculations
    probability_signal_given_bit_one = np.zeros(len(received_signal))
    probability_signal_given_bit_zero = np.zeros(len(received_signal))

    for i in range(len(received_signal)):
        probability_signal_given_bit_one[i] = (noise_ratio ** ((received_signal[i] + 1) % 2) * 
                                               (1 - noise_ratio) ** ((received_signal[i] + 2) % 2))
        probability_signal_given_bit_zero[i] = (noise_ratio ** (received_signal[i]) * 
                                                (1 - noise_ratio) ** ((received_signal[i] + 1) % 2))

    # Initialize message matrix
    message_matrix = np.zeros((num_rows, num_cols))

    for row in range(num_rows):
        for col in range(num_cols):
            message_matrix[row, col] = (np.log([probability_signal_given_bit_zero[col] / 
                                                probability_signal_given_bit_one[col]]))
    
    for iteration in range(maximum_iterations):
        
        # Factor to variable message matrix
        message_factor_variable = np.zeros((num_rows, num_cols))

        for index in range(len(bit_check_indices)):
            new_row = np.zeros(num_cols)
            product = message_matrix[index, (bit_check_indices[index])]
            
            for i in range(len(product)):
                product_excluding_self = np.hstack((product[0:i], product[i + 1: len(product)]))
                product_result = np.prod(np.tanh(product_excluding_self / 2))
                log_value = (1 + product_result) / (1 - product_result)
                new_row[bit_check_indices[index][i]] = np.log(log_value)
            
            message_factor_variable[index, :] = new_row

        # Calculate log likelihood of posterior
        posterior_log_likelihood = np.sum(message_factor_variable, axis=0)
        
        for i in range(num_cols):
            posterior_log_likelihood[i] += (np.log([probability_signal_given_bit_zero[i] / 
                                                    probability_signal_given_bit_one[i]]))
        
        # Update prediction
        decoded_signal = np.zeros(num_cols)
        for i in range(num_cols):
            decoded_signal[i] = 0 if posterior_log_likelihood[i] > 0 else 1
            
        # Test for parity conditions
        parity_test = np.matmul(parity_check_matrix, decoded_signal.T) % 2

        # Check if parity conditions are met or maximum iterations reached
        if iteration == maximum_iterations or np.all(parity_test == 0):
            decoding_success = 0
            break
        else:
            # Update variable to factor message
            for col in range(num_cols):
                for row in range(num_rows):
                    message_matrix[row, col] = posterior_log_likelihood[col] - message_factor_variable[row, col]
    
    return decoding_success, iteration + 1, decoded_signal

In [11]:
success, iter, decoded = decode(h1, y1, noise_ratio=0.1, maximum_iterations=20)

In [12]:
print("success: " + str(success), "\n", "iteration: " + str(iter), "\n", "decoded result: " + "\n" + str(decoded))

success: 0 
 iteration: 8 
 decoded result: 
[0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1.
 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.
 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0.

In [13]:
x = np.reshape(decoded.astype(int), (125, 8)).astype(str)

b = [''.join(row) for row in x]

message = ''.join(chr(int(binary, 2)) for binary in b[:31])

print(message)

Happy Holidays! Dmitry&David :)
